In [1]:
from lsst.daf.butler import Butler
from lsst.obs.base import DefineVisitsTask, Instrument


In [2]:
# I think we're talking about ComCam data, right?
# This collections list would work at NCSA, but I suspect it will need adjustment
# elsewhere. Needs to include both the raws and the `camera` dataset.
butler = Butler(
    "/repo/LSSTComCam/",
    collections=["LSSTComCam/raw/all"],
    writeable=True
)


In [3]:
intra_exposure_id = 2022061700011
extra_exposure_id = 2022061700012

In [4]:
# Replace "???" below by a query string that corresponds to all wavefront observations.
exposure_data_ids = set(
    butler.registry.queryDataIds(
        [
            "exposure"
        ], 
        where=f"exposure IN ({intra_exposure_id}, {extra_exposure_id})"
    )
)

In [5]:
exposure_data_ids

{{instrument: 'LSSTComCam', exposure: 2022061700011, ...},
 {instrument: 'LSSTComCam', exposure: 2022061700012, ...}}

In [6]:
instrument = Instrument.fromName("LSSTComCam", registry=butler.registry)

In [7]:
config = DefineVisitsTask.ConfigClass()

In [8]:
instrument.applyConfigOverrides(DefineVisitsTask._DefaultName, config)

In [9]:
config.groupExposures.name = "one-to-one"

In [10]:
task = DefineVisitsTask(config=config, butler=butler)

In [11]:
task.run(exposure_data_ids)

In [14]:
data_id = {'instrument': 'LSSTComCam', 'detector': 0, 'exposure': 2022061700012}

In [15]:
metadata = butler.get('raw.visitInfo', dataId=data_id, collections=["LSSTComCam/raw/all"])
print(metadata)

VisitInfo(exposureId=2022061700012000, exposureTime=15, darkTime=18.1156, date=2022-06-17T19:49:23.661502482, UT1=nan, ERA=3.5422 rad, boresightRaDec=(307.0781171669, -87.4721945038), boresightAzAlt=(nan, +nan), boresightAirmass=2.1467, boresightRotAngle=8.62307e-318 rad, rotType=1, observatory=-30.2446N, -70.7494E  2663, weather=Weather(nan, nan, nan), instrumentLabel='LSSTComCam', id=2022061700012)
